# Evaluation Summary Plots/Anaylisis

Plots/Analysis for a single evaluation for a single experiment. For now, only the default evaluation is considered.

## Experiments selection

In [ ]:
# Common imports.
from pathlib import Path

%matplotlib widget
import base

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import ipywidgets

import dfaas_env
import dfaas_utils
import dfaas_upperbound

Select one or more experiments to view.

**WARNING**: If multiple experiments are selected, they must share the same number of evaluations!

In [ ]:
experiments = base.get_experiments("/home/emanuele/marl-dfaas/results")

# Show the name as the portion of the path after "results",
# but anyway the values are full Path objects.
exp_select = ipywidgets.SelectMultiple(
    options=experiments,
    index=[0],
    description="Experiment(s):",
    style={"description_width": "initial"},
    layout=ipywidgets.Layout(width="70%"),
)

ipywidgets.AppLayout(center=exp_select)

## Experiments loading

In [ ]:
exps_dir = exp_select.value
assert len(exps_dir) > 0, "must select at least one experiment"

# Preload the data (evaluation.json file) for all selected experiments.
raw_eval_data = {}
for exp_dir in exps_dir:
    raw_eval_data[exp_dir] = dfaas_utils.parse_result_file(exp_dir / "evaluation.json")

# Create the reference environment based on DFaaS.
for exp_dir in exps_dir:
    env = base.get_env(exp_dir)
    if env.__class__ == dfaas_env.DFaaS:
        break

# At least one experiment must be of type DFaaS. SingleDFaaS is used only as
# reference.
assert env.__class__ == dfaas_env.DFaaS, f"{env.__class__}"

print("Selected experiments:")
for exp_dir in exps_dir:
    print(f"  - {exp_dir.name}")

## Reward (all episodes)

In [ ]:
# Common functions for average reward data.


def get_reward_data(raw_eval_data):
    final_data = {"episodes": None}

    for exp_dir, eval_data in raw_eval_data.items():
        env = base.get_env(exp_dir)
        if env.__class__ == dfaas_upperbound.SingleDFaaS:
            # The upperbound is not used for the reward plots.
            continue

        episodes = eval_data[0]["env_runners"]["episodes_this_iter"]
        if final_data["episodes"] is None:
            final_data["episodes"] = episodes
        else:
            assert (
                episodes == final_data["episodes"]
            ), "All evaluation experiments must have the same number of episodes!"

        exp_data = {"all": eval_data[0]["env_runners"]["hist_stats"]["episode_reward"]}
        for agent in env.agents:
            exp_data[agent] = eval_data[0]["env_runners"]["hist_stats"][
                f"policy_policy_{agent}_reward"
            ]

        final_data[exp_dir] = exp_data

    return final_data


reward_data = get_reward_data(raw_eval_data)

### Total per episode

#### All agents

In [ ]:
def make_reward_plot_all_agents():
    plt.close(fig="reward_total")
    fig = plt.figure(num="reward_total", layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    ax.axhline(
        y=env.max_steps * len(env.agents), color="red", linestyle="--", label="Limit"
    )
    for exp_dir in exps_dir:
        if base.get_env(exp_dir).__class__ == dfaas_upperbound.SingleDFaaS:
            # The upperbound is not used for the reward plots.
            continue

        stemlines = ax.stem(
            reward_data[exp_dir]["all"], linefmt="-", basefmt="None", label=exp_dir.name
        )
        plt.setp(stemlines[1], "alpha", 0)  # Set the linefmt to be transparent.

    ax.set_title("Total reward per episode (all agents)")

    ax.set_ylabel("Reward per episode")
    ax.set_ylim(bottom=0, top=env.max_steps * len(env.agents) * 1.05)

    ax.set_xlabel("Episode")

    ax.legend()
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.


make_reward_plot_all_agents()

#### Single agents

In [ ]:
def make_reward_plot_single_agents():
    for agent in env.agents:
        plt.close(fig=f"reward_total_{agent}")
        fig = plt.figure(num=f"reward_total_{agent}", layout="constrained")
        fig.canvas.header_visible = False
        ax = fig.subplots()

        ax.axhline(y=env.max_steps, color="red", linestyle="--", label="Limit")
        for exp_dir in exps_dir:
            if base.get_env(exp_dir).__class__ == dfaas_upperbound.SingleDFaaS:
                # The upperbound is not used for the reward plots.
                continue
            stemlines = ax.stem(
                reward_data[exp_dir][agent],
                linefmt="-",
                basefmt="None",
                label=exp_dir.name,
            )
            plt.setp(stemlines[1], "alpha", 0)  # Set the linefmt to be transparent.

        ax.set_title(f"Total reward per episode ({agent = })")

        ax.set_ylabel("Reward per episode")
        ax.set_ylim(bottom=0, top=env.max_steps * 1.05)

        ax.set_xlabel("Episode")

        ax.legend()
        ax.grid(axis="both")
        ax.set_axisbelow(True)  # By default the axis is over the content.


make_reward_plot_single_agents()

## Processed requests

In [ ]:
# Common functions for processed requests.


def get_processed_requests_data_upperbound(env, eval_data):
    episodes = eval_data[0]["env_runners"]["episodes_this_iter"]
    data = {"input_reqs": np.empty(episodes), "processed_reqs": np.empty(episodes)}

    for epi_idx in range(episodes):
        input_reqs = np.sum(
            eval_data[0]["env_runners"]["hist_stats"]["observation_input_requests"][
                epi_idx
            ]
        )
        processed_local = np.sum(
            eval_data[0]["env_runners"]["hist_stats"]["processed_local"][epi_idx]
        )

        data["input_reqs"][epi_idx] = input_reqs
        data["processed_reqs"][epi_idx] = processed_local

    return data


def get_processed_requests_data(raw_eval_data):
    final_data = {"episodes": None}

    for exp_dir, eval_data in raw_eval_data.items():
        episodes = eval_data[0]["env_runners"]["episodes_this_iter"]
        if final_data["episodes"] is None:
            final_data["episodes"] = episodes
        else:
            assert (
                episodes == final_data["episodes"]
            ), "All evaluation experiments must have the same number of episodes!"

        env = base.get_env(exp_dir)
        if env.__class__ == dfaas_upperbound.SingleDFaaS:
            # The upperbound data extraction is different from the normal flow.
            final_data[exp_dir] = get_processed_requests_data_upperbound(env, eval_data)
            continue

        final_data[exp_dir] = {}
        for agent in ["all"] + env.agents:
            final_data[exp_dir][agent] = {}
            final_data[exp_dir][agent]["input_reqs"] = np.empty(episodes)
            final_data[exp_dir][agent]["processed_reqs"] = np.empty(episodes)
            final_data[exp_dir][agent]["processed_forwarded_reqs"] = np.empty(episodes)

        for epi_idx in range(episodes):
            input_reqs, processed_reqs, processed_forwarded_reqs = 0, 0, 0

            for agent in env.agents:
                agent_input_reqs = np.sum(
                    eval_data[0]["env_runners"]["hist_stats"][
                        "observation_input_requests"
                    ][epi_idx][agent]
                )
                agent_processed_reqs = np.sum(
                    eval_data[0]["env_runners"]["hist_stats"]["processed_local"][
                        epi_idx
                    ][agent]
                )

                try:
                    agent_processed_forward = np.sum(
                        eval_data[0]["env_runners"]["hist_stats"][
                            "processed_local_forward"
                        ][epi_idx][agent]
                    )
                except KeyError:
                    # May be missing if the agent did not receive any forwarded request.
                    agent_processed_forward = 0

                final_data[exp_dir][agent]["input_reqs"][epi_idx] = agent_input_reqs
                final_data[exp_dir][agent]["processed_reqs"][
                    epi_idx
                ] = agent_processed_reqs
                final_data[exp_dir][agent]["processed_forwarded_reqs"][
                    epi_idx
                ] = agent_processed_forward
                input_reqs += agent_input_reqs
                processed_reqs += agent_processed_reqs
                processed_forwarded_reqs += agent_processed_forward

            final_data[exp_dir]["all"]["input_reqs"][epi_idx] = input_reqs
            final_data[exp_dir]["all"]["processed_reqs"][epi_idx] = processed_reqs
            final_data[exp_dir]["all"]["processed_forwarded_reqs"][
                epi_idx
            ] = processed_forwarded_reqs

    return final_data


processed_reqs_data = get_processed_requests_data(raw_eval_data)

### Average processed requests per episode

#### All agents

In [ ]:
def make_processed_reqs_plot_all_agents():
    plt.close(fig="processed_reqs_all_agents")
    fig = plt.figure(num="processed_reqs_all_agents", layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    data = processed_reqs_data  # Alias for better readability.
    for exp_dir in exps_dir:
        if base.get_env(exp_dir).__class__ == dfaas_upperbound.SingleDFaaS:
            # This is the upperbound case.
            ratios = data[exp_dir]["processed_reqs"] / data[exp_dir]["input_reqs"]
            ax.plot(ratios, label="Upperbound")
            continue

        ratios = (
            data[exp_dir]["all"]["processed_reqs"] / data[exp_dir]["all"]["input_reqs"]
        )
        stemlines = ax.stem(ratios, linefmt="-", basefmt="None", label=exp_dir.name)
        plt.setp(stemlines[1], "alpha", 0)  # Set the linefmt to be transparent.

    ax.set_title("Processed requests per episode (all agents)")

    ax.set_ylabel("Requests")
    ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
    ax.set_ylim(0, 1)

    ax.set_xlabel("Episode")

    ax.legend(loc="lower center")
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.


make_processed_reqs_plot_all_agents()

#### Single agents

In [ ]:
def make_processed_reqs_plot_single_agents():
    for agent in env.agents:
        plt.close(fig=f"processed_reqs_{agent}")
        fig = plt.figure(num=f"processed_reqs_{agent}", layout="constrained")
        fig.canvas.header_visible = False
        ax = fig.subplots()

        data = processed_reqs_data  # Alias for better readability.
        for exp_dir in exps_dir:
            if base.get_env(exp_dir).__class__ == dfaas_upperbound.SingleDFaaS:
                # The upperbound is valid only for global view (all agents).
                continue

            ratios = (
                data[exp_dir][agent]["processed_reqs"]
                / data[exp_dir][agent]["input_reqs"]
            )
            stemlines = ax.stem(ratios, linefmt="-", basefmt="None", label=exp_dir.name)
            plt.setp(stemlines[1], "alpha", 0)  # Set the linefmt to be transparent.

        ax.set_title(f"Processed requests per episode ({agent = })")

        ax.set_ylabel("Requests")
        ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
        ax.set_ylim(0, 1)

        ax.set_xlabel("Episode")

        ax.legend()
        ax.grid(axis="both")
        ax.set_axisbelow(True)  # By default the axis is over the content.


make_processed_reqs_plot_single_agents()

### Average processed forwarded requests per episode

#### All agents

In [ ]:
def make_processed_fw_reqs_plot_all_agents():
    plt.close(fig="avg_processed_fw_reqs")
    fig = plt.figure(num="avg_processed_fw_reqs", layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    data = processed_reqs_data  # Alias for better readability.
    for exp_dir in exps_dir:
        if base.get_env(exp_dir).__class__ == dfaas_upperbound.SingleDFaaS:
            # The upperbound is valid only for total processed requests.
            continue

        ratios_forwarded = (
            data[exp_dir]["all"]["processed_forwarded_reqs"]
            / data[exp_dir]["all"]["processed_reqs"]
        )
        stemlines = ax.stem(
            ratios_forwarded, linefmt="-", basefmt="None", label=exp_dir.name
        )
        plt.setp(stemlines[1], "alpha", 0)  # Set the linefmt to be transparent.

    ax.set_title("Average processed forwarded requests per episode (all agents)")

    ax.set_ylabel("Requests")
    ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
    ax.set_ylim(0, 1)

    ax.set_xlabel("Episodes")

    ax.legend()
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.


make_processed_fw_reqs_plot_all_agents()

#### Single agents

In [ ]:
def make_processed_fw_reqs_plot_single_agents():
    for agent in env.agents:
        plt.close(fig=f"processed_fw_reqs_{agent}")
        fig = plt.figure(num=f"processed_fw_reqs_{agent}", layout="constrained")
        fig.canvas.header_visible = False
        ax = fig.subplots()

        data = processed_reqs_data  # Alias for better readability.
        for exp_dir in exps_dir:
            if base.get_env(exp_dir).__class__ == dfaas_upperbound.SingleDFaaS:
                # The upperbound is valid only for total processed requests.
                continue

            ratios_forwarded = (
                data[exp_dir][agent]["processed_forwarded_reqs"]
                / data[exp_dir][agent]["processed_reqs"]
            )
            stemlines = ax.stem(
                ratios_forwarded, linefmt="-", basefmt="None", label=exp_dir.name
            )
            plt.setp(stemlines[1], "alpha", 0)  # Set the linefmt to be transparent.

        ax.set_title(f"Average processed forwarded requests per episode ({agent = })")

        ax.set_ylabel("Requests")
        ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
        ax.set_ylim(0, 1)

        ax.set_xlabel("Episodes")

        ax.legend()
        ax.grid(axis="both")
        ax.set_axisbelow(True)  # By default the axis is over the content.


make_processed_fw_reqs_plot_single_agents()

## Queue size

In [ ]:
# Common functions for queue.


def get_queue_data_upperbound(env, eval_data):
    episodes = eval_data[0]["env_runners"]["episodes_this_iter"]
    data = {
        "queue_size": np.empty(episodes),
        "queue_full": np.empty(episodes),
        "queue_size_std": np.empty(episodes),
    }

    for epi_idx in range(episodes):
        queue = np.array(
            eval_data[0]["env_runners"]["hist_stats"]["queue_size"][epi_idx]
        )

        data["queue_size"][epi_idx] = np.mean(queue)
        data["queue_size_std"][epi_idx] = np.std(queue)
        data["queue_full"][epi_idx] = np.where(queue == env.queue_capacity)[0].size

    return data


def get_queue_data(raw_eval_data):
    final_data = {"episodes": None}

    for exp_dir, eval_data in raw_eval_data.items():
        episodes = eval_data[0]["env_runners"]["episodes_this_iter"]
        if final_data["episodes"] is None:
            final_data["episodes"] = episodes
        else:
            assert (
                episodes == final_data["episodes"]
            ), "All evaluation experiments must have the same number of episodes!"

        env = base.get_env(exp_dir)
        if env.__class__ == dfaas_upperbound.SingleDFaaS:
            # The upperbound data extraction is different from the normal flow.
            final_data[exp_dir] = get_queue_data_upperbound(env, eval_data)
            continue

        final_data[exp_dir] = {}
        for agent in ["all"] + env.agents:
            final_data[exp_dir][agent] = {}
            final_data[exp_dir][agent]["queue_size_mean"] = np.empty(episodes)
            final_data[exp_dir][agent]["queue_size_std"] = np.empty(episodes)
            final_data[exp_dir][agent]["queue_full"] = np.empty(episodes)

        for epi_idx in range(episodes):
            queue_all = np.empty(0)

            for agent in env.agents:
                queue = np.array(
                    eval_data[0]["env_runners"]["hist_stats"]["queue_size"][epi_idx][
                        agent
                    ]
                )
                queue_all = np.concatenate([queue_all, queue])

                final_data[exp_dir][agent]["queue_size_mean"][epi_idx] = np.mean(queue)
                final_data[exp_dir][agent]["queue_size_std"][epi_idx] = np.std(queue)

                # Count the number of occurencies where queue is full
                final_data[exp_dir][agent]["queue_full"][epi_idx] = np.where(
                    queue == env.queue_capacity
                )[0].size

            final_data[exp_dir]["all"]["queue_size_mean"][epi_idx] = np.mean(queue_all)
            final_data[exp_dir]["all"]["queue_size_std"][epi_idx] = np.std(queue_all)
            final_data[exp_dir]["all"]["queue_full"][epi_idx] = np.where(
                queue == env.queue_capacity
            )[0].size

    return final_data


queue_data = get_queue_data(raw_eval_data)

### Average queue size per episode

#### All agents

In [ ]:
def make_queue_size_plot_all_agents():
    plt.close(fig="queue_size")
    fig = plt.figure(num="queue_size", layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    for exp_dir in exps_dir:
        if base.get_env(exp_dir).__class__ == dfaas_upperbound.SingleDFaaS:
            # This is the upperbound case.
            stemlines = ax.stem(
                queue_data[exp_dir]["queue_size"] / env.queue_capacity,
                linefmt="-",
                basefmt="None",
                label="Upperbound",
            )
            plt.setp(stemlines[1], "alpha", 0)  # Set the linefmt to be transparent.
            continue

        stemlines = ax.stem(
            queue_data[exp_dir]["all"]["queue_size_mean"] / env.queue_capacity,
            linefmt="-",
            basefmt="None",
            label=exp_dir.name,
        )
        plt.setp(stemlines[1], "alpha", 0)  # Set the linefmt to be transparent.

    ax.set_title("Average queue size per episode (all agents)")

    ax.set_ylabel("Requests in queue")
    ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
    ax.set_ylim(0, 1)

    ax.set_xlabel("Episode")

    ax.legend()
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.


make_queue_size_plot_all_agents()

#### Single agents

In [ ]:
def make_queue_size_plot_single_agents():
    for agent in env.agents:
        plt.close(fig=f"queue_size_{agent}")
        fig = plt.figure(num=f"queue_size_{agent}", layout="constrained")
        fig.canvas.header_visible = False
        ax = fig.subplots()

        for exp_dir in exps_dir:
            if base.get_env(exp_dir).__class__ == dfaas_upperbound.SingleDFaaS:
                # The upperbound is valid only for all agents.
                continue

            stemlines = ax.stem(
                queue_data[exp_dir][agent]["queue_size_mean"] / env.queue_capacity,
                linefmt="-",
                basefmt="None",
                label=exp_dir.name,
            )
            plt.setp(stemlines[1], "alpha", 0)  # Set the linefmt to be transparent.

        ax.set_title(f"Average queue size per episode ({agent = })")

        ax.set_ylabel("Requests in queue")
        ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
        ax.set_ylim(0, 1)

        ax.set_xlabel("Iteration")

        ax.legend(loc="lower center")
        ax.grid(axis="both")
        ax.set_axisbelow(True)  # By default the axis is over the content.


make_queue_size_plot_single_agents()

### Average queue full per episode

#### All agents

In [ ]:
def make_queue_full_plot_all_agents():
    plt.close(fig="avg_queue_full")
    fig = plt.figure(num="avg_queue_full", layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    ax.axhline(
        y=env.max_steps * len(env.agents),
        color="red",
        linestyle="--",
        label="Upper Limit",
    )
    for exp_dir in exps_dir:
        if base.get_env(exp_dir).__class__ == dfaas_upperbound.SingleDFaaS:
            # This is the upperbound case.
            stemlines = ax.stem(
                queue_data[exp_dir]["queue_full"],
                linefmt="-",
                basefmt="None",
                label="Upperbound",
            )
            plt.setp(stemlines[1], "alpha", 0)  # Set the linefmt to be transparent.
            continue

        stemlines = ax.stem(
            queue_data[exp_dir]["all"]["queue_full"],
            linefmt="-",
            basefmt="None",
            label=exp_dir.name,
        )
        plt.setp(stemlines[1], "alpha", 0)  # Set the linefmt to be transparent.

    ax.set_title("Times the queue is full per episode (all agents)")

    ax.set_ylabel("Queue full")
    ax.set_ylim(bottom=0, top=env.max_steps * len(env.agents) * 1.05)

    ax.set_xlabel("Iteration")

    ax.legend()
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.


make_queue_full_plot_all_agents()

#### Single agents

In [ ]:
def make_queue_full_plot_single_agents():
    for agent in env.agents:
        plt.close(fig=f"avg_queue_full_{agent}")
        fig = plt.figure(num=f"avg_queue_full_{agent}", layout="constrained")
        fig.canvas.header_visible = False
        ax = fig.subplots()

        ax.axhline(
            y=env.max_steps,
            color="red",
            linestyle="--",
            label="Upper Limit",
        )
        for exp_dir in exps_dir:
            if base.get_env(exp_dir).__class__ == dfaas_upperbound.SingleDFaaS:
                # The upperbound is valid only for all agents.
                continue

            stemlines = ax.stem(
                queue_data[exp_dir][agent]["queue_full"],
                linefmt="-",
                basefmt="None",
                label=exp_dir.name,
            )
            plt.setp(stemlines[1], "alpha", 0)  # Set the linefmt to be transparent.

        ax.set_title(f"Times the queue is full per episode ({agent = })")

        ax.set_ylabel("Queue full")
        ax.set_ylim(bottom=0, top=env.max_steps * 1.05)

        ax.set_xlabel("Iteration")

        ax.legend()
        ax.grid(axis="both")
        ax.set_axisbelow(True)  # By default the axis is over the content.


make_queue_full_plot_single_agents()